## Avaliando [Crash Course](https://mxnet.apache.org/versions/master/api/python/docs/tutorials/getting-started/crash-course/2-create-nn.html)


### Step2

In [1]:
import mxnet as mx
from mxnet import np, npx
from mxnet.gluon import nn

npx.set_np()

## Step 2 - Create neural network
Uma camada com 3 entradas e 5 saidas, com função de ativação "RELU"


In [2]:
layer=nn.Dense(5,in_units=3,activation='relu')
layer

Dense(3 -> 5, Activation(relu))

In [3]:
layer.initialize()

[11:17:43] /home/mertins/Desenvolvimento/C/Terceiros/mxnet/src/storage/storage.cc:202: Using Pooled (Naive) StorageManager for CPU


#### dez linhas com três parametros

In [4]:
x=np.random.uniform(-1,1,(10,3))
x

array([[-0.88657403,  0.9273255 , -0.45468742],
       [-0.23311698, -0.04466975,  0.5834501 ],
       [ 0.62433743,  0.0577898 , -0.04004568],
       [ 0.13608909, -0.21443039,  0.8511933 ],
       [ 0.6721575 , -0.8579279 , -0.32520765],
       [-0.8257414 ,  0.2963438 , -0.9595632 ],
       [-0.2635169 ,  0.6652397 ,  0.91431034],
       [ 0.5563135 , -0.7192985 ,  0.7400243 ],
       [ 0.74017453,  0.95723665, -0.05278391],
       [ 0.59831715,  0.60182154, -0.07704127]])

In [5]:
layer(x)

array([[0.        , 0.        , 0.04437108, 0.01096384, 0.        ],
       [0.01540359, 0.01735983, 0.        , 0.        , 0.03530429],
       [0.00381139, 0.02891595, 0.0071606 , 0.01262893, 0.        ],
       [0.02378628, 0.04612946, 0.        , 0.        , 0.05157538],
       [0.        , 0.00375629, 0.        , 0.        , 0.00060787],
       [0.        , 0.        , 0.01947428, 0.00730047, 0.        ],
       [0.03439108, 0.04268821, 0.02091249, 0.        , 0.03359453],
       [0.01674652, 0.05354869, 0.        , 0.        , 0.05611669],
       [0.01591048, 0.04680086, 0.05175146, 0.03321034, 0.        ],
       [0.00959048, 0.03363482, 0.03384075, 0.02388959, 0.        ]])

In [6]:
layer.params

{'weight': Parameter (shape=(5, 3), dtype=float32),
 'bias': Parameter (shape=(5,), dtype=float32)}

#### pesos e bias de entrada de cada elemento da camada interna de 5 neuronios.

In [7]:
layer.weight.data()

array([[ 0.0068339 ,  0.01299825,  0.0301265 ],
       [ 0.04819721,  0.01438687,  0.05011239],
       [ 0.00628365,  0.04861524, -0.01068833],
       [ 0.01729892,  0.02042518, -0.01618656],
       [-0.00873779, -0.02834515,  0.05484822]])

In [8]:
layer.bias.data()


array([0., 0., 0., 0., 0.])

Chain layers into a neural network using nn.Sequential

Sequential é a ligação de camadas onde a saída de uma camada é a entrada de outra

In [9]:
net = nn.Sequential()

net.add(nn.Dense(5,in_units=3,activation='relu'),nn.Dense(25,activation='relu'),nn.Dense(2))

net

Sequential(
  (0): Dense(3 -> 5, Activation(relu))
  (1): Dense(-1 -> 25, Activation(relu))
  (2): Dense(-1 -> 2, linear)
)

In [10]:
net[1]

Dense(-1 -> 25, Activation(relu))

#### Custom neural network architecture flexibly

In [11]:
class Net(nn.Block):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x

In [12]:
class MLP(nn.Block):
    def __init__(self):
        super().__init__()
        self.dense1=nn.Dense(5,activation='relu')
        self.dense2=nn.Dense(25,activation='relu')
        self.dense3=nn.Dense(2)
    def forward(self,x):
        layer1=self.dense1(x)
        layer2=self.dense2(layer1)
        layer3=self.dense3(layer2)
        return layer3

In [13]:
net=MLP()
net

MLP(
  (dense1): Dense(-1 -> 5, Activation(relu))
  (dense2): Dense(-1 -> 25, Activation(relu))
  (dense3): Dense(-1 -> 2, linear)
)

In [14]:
net.dense1.params

{'weight': Parameter (shape=(5, -1), dtype=float32),
 'bias': Parameter (shape=(5,), dtype=float32)}

#### Creating custom layers using Parameters

In [15]:
from mxnet.gluon import Parameter

weight=Parameter("custom_parameter_weight",shape=(5,-1))
bias=Parameter("custom_parameter_bias",shape=(5,-1))

weight,bias

(Parameter (shape=(5, -1), dtype=<class 'numpy.float32'>),
 Parameter (shape=(5, -1), dtype=<class 'numpy.float32'>))

Custom layer com função linear sem função de ativação. 

w*x+b

In [16]:
class custom_layer(nn.Block):
    def __init__(self,out_units, in_units=0):
        super().__init__()
        self.weight=Parameter("weight",shape=(in_units,out_units),allow_deferred_init=True)
        self.bias=Parameter("bias",shape=(out_units,),allow_deferred_init=True)
    def forward(self,x):
        return np.dot(x,self.weight.data())+self.bias.data()

In [17]:
dense=custom_layer(3,in_units=5)
dense.initialize()
dense(np.random.uniform(size=(4,5)))

array([[-0.02481367,  0.007271  ,  0.06815495],
       [-0.0675921 , -0.06589589,  0.04664002],
       [-0.05190407, -0.01203752,  0.05761838],
       [-0.03687233, -0.00421239,  0.06642268]])

### [LeNet](http://yann.lecun.com/exdb/lenet/)

In [ ]:
class LeNet(nn.Block):
    def __init__(self):
        self.conv1=nn.Conv2D(channels=6,kernel_size=3,activation='relu')
        self.pool1=nn.MaxPool2D(pool_size=2,strides=2)
        self.conv2=nn.Conv2D(channels=16,kernel_size=3,activation='relu')
        